In [1]:
import os
import numpy as np
#from train_lstm_oh import build_model

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.models import Model

from keras.layers.wrappers import TimeDistributed, Bidirectional
from keras.layers.recurrent import GRU,LSTM
from keras.layers import Input
from keras.layers.convolutional import ZeroPadding3D, Conv2D
from keras import regularizers
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Lambda, Dropout, Flatten, Dense, Activation
from keras.optimizers import Adam
from keras import backend as K
from keras.applications.vgg16 import VGG16

Using TensorFlow backend.


In [2]:
CURRENT_PATH = os.getcwd()
DATA_PATH = CURRENT_PATH

In [3]:
# load test data
x_path = DATA_PATH + '/X.npz'
y_path = DATA_PATH + '/y.npz'
x_s = np.load(x_path)['x']
y_s = np.load(y_path)['y']
x_train, x_test, y_train, y_test = train_test_split(x_s, y_s, test_size=0.2, shuffle=True)

In [4]:
def build_model(input_size, output_size=28):
    #########################################################
    # Model: conv2d + conv2d + maxpool + dropout + conv2d +
    #  bidirectional gru + dense + softmax + ctc loss
    #########################################################

    ## input_size: placeholder in Keras
    #  shape: (None, seq_size = 20, height = 50, width = 100, channels = 3)
    # if K.image_data_format() == 'channels_first':
    #     input_size = (self.img_c, self.frames_n, self.img_w, self.img_h)
    # else:
    #     input_size = (self.frames_n, self.img_w, self.img_h, self.img_c)

    # self.input_data = Input(name='the_input', shape=input_size, dtype='float32')
    input_data = Input(name='the_input', shape=input_size, dtype='float32')
    
    ## padding used on the height and width before convolving
    #  shape:(None, 20, 54, 104, 3)
    model = Sequential()
    model.add(ZeroPadding3D(padding=(0, 2, 2),input_shape = (input_size), name='padding1'))

    ## 2D Convolution on each time sequence, relu activation
    #  shape 1st conv: (None, 20, 27, 52, 32)
    #  shape 2nd conv: (None, 20, 14, 26, 32)
    model.add(TimeDistributed(Conv2D(filters=34, kernel_size=(3, 3), kernel_initializer='he_normal', strides=(2, 2), padding='same', activation='relu')))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='max1')))

    model.add(TimeDistributed(Conv2D(filters=64, kernel_size=(3, 3), kernel_initializer='he_normal', strides=(2, 2), padding='same', activation='relu')))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='max2')))
    model.add(Dropout(0))

    ## 2D Convolution on each time sequence, relu activation
    #  shape 1st conv: (None, 20, 4, 7, 4)
    model.add(TimeDistributed(Conv2D(filters=96, kernel_size=(3, 3), kernel_initializer='he_normal', strides=(2, 2), padding='same', activation='relu')))

    ## Flatten to gru
    #  shape: (None, 20, 112)
    model.add(TimeDistributed(Flatten()))
    model.add(Dropout(0.0))
    ## Bidirectional gru
    #  shape: (None, 20, 512)
    # x_lstm = LSTM(256, return_sequences=True, kernel_initializer='Orthogonal', name='lstm1')(input_lstm)
    # x_lstm = LSTM(256, return_sequences=True, kernel_initializer='Orthogonal', name='lstm2')(x_lstm)
    model.add(Bidirectional(GRU(256, return_sequences=True, kernel_initializer='Orthogonal', name='gru1'),
                           merge_mode='concat'))
    model.add(Bidirectional(GRU(256, return_sequences=True, kernel_initializer='Orthogonal', name='gru2'),
                           merge_mode='concat'))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_normal', name='fc1', kernel_regularizer=regularizers.l2(0.0)))
    model.add(Dense(output_size, kernel_initializer='he_normal', name='dense2'))

    ## prepare input for ctc loss
    model.add(Activation('softmax', name='softmax'))
    #loss = CTC('ctc', [y_pred, labels, input_length, label_length])
    model.summary()

    return model
def build_model_vgg16(input_size, output_size=28):
    input_data = Input(name='the_input', shape=input_size, dtype='float32')

    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=input_size[1:], pooling='max')
    x = TimeDistributed(vgg16)(input_data)
    x = TimeDistributed(Flatten())(x)

    # let's add a fully-connected layer
    x = Bidirectional(GRU(256, return_sequences=True, name='gru1'), merge_mode='concat')(x)
    x = Bidirectional(GRU(256, return_sequences=True, name='gru1'), merge_mode='concat')(x)
    x = Flatten()(x)

    x = Dense(4096, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = Dropout(0.2)(x)
    predictions = Dense(output_size, activation='softmax')(x)

    # this is the model we will train
    model = Model(inputs=input_data, outputs=predictions)

    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional InceptionV3 layers
    for layer in vgg16.layers:
       layer.trainable = False
    model.summary()
    return model
def train(model, x_train, y_train, batch_size=256, epochs=100, val_train_ratio=0.2, start_epoch=0):
    ##
    # Train model, typically will train for each speaker
    ## padding the labels
    # max_string_len = 10
    # if y_train.shape[1] != max_string_len:
    #     y_train = pad_labels(y_train, max_string_len)

    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

    if start_epoch > 0:
        weight_file = os.path.join(CURRENT_PATH, "models/model_lstm_oh.h5")
        model.load_weights(weight_file)
    ## callbacks when each epoch ends
    #  This will ouput character error rate which
    #  compares each predicted word with source word.
    #  TODO: results file need to be implemented
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_split=val_train_ratio,
                        shuffle=True,
                        initial_epoch=start_epoch,
                        verbose=1)
    return history

In [5]:
# load model
# if already has a model do not train it, this is just for test
# model = build_model_vgg16(x_test.shape[1:], output_size=y_test.shape[1])
# history = train(model, x_train, y_train, batch_size=256, epochs=20,
#                     start_epoch=0)
# model.save('model_lstm_oh.h5')

In [6]:
# change model dir here
model_dir = 'model_vgg16-gru-fc4096-lr0.0001.h5'
model = build_model_vgg16(x_test.shape[1:], output_size=y_test.shape[1])

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
weight_file = os.path.join(CURRENT_PATH, model_dir)
model.load_weights(weight_file)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       (None, 21, 50, 100, 3)    0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 21, 512)           14714688  
_________________________________________________________________
time_distributed_2 (TimeDist (None, 21, 512)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 21, 512)           1181184   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 21, 512)           1181184   
_________________________________________________________________
flatten_2 (Flatten)          (None, 10752)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              44044288  
__________

In [23]:
# choose one sample to test
ind = 4
preds = model.predict(np.expand_dims(x_test[ind],0))
pred_label = np.argmax(preds)
least_label = np.argmin(preds)
true_label = np.argmax(y_test[ind])
print("true label is: ",true_label, "predict label is :", pred_label )
print("true label is: ",true_label, "least label is :", least_label )
print("probabilities: ",preds)

('true label is: ', 3, 'predict label is :', 3)
('true label is: ', 3, 'least label is :', 27)
('probabilities: ', array([[  6.14955352e-05,   7.16151681e-06,   7.50396357e-05,
          9.56085503e-01,   2.63631321e-03,   2.04445896e-04,
          2.23993342e-02,   5.14794410e-05,   5.79983927e-04,
          3.21533502e-04,   3.15910159e-03,   4.54713663e-05,
          4.02157457e-06,   1.28817959e-07,   5.72489853e-06,
          1.94282678e-04,   2.24892956e-05,   2.37194399e-06,
          7.43483906e-05,   1.54822101e-06,   1.94282111e-04,
          4.06555091e-06,   6.66772803e-06,   7.48415550e-06,
          6.60496808e-05,   1.29408249e-07,   6.08751507e-05,
          1.22314034e-07,   2.34490471e-06,   1.01880431e-02,
          8.16547952e-04,   1.24941964e-03,   3.38988184e-06,
          1.29785946e-07,   1.45517930e-04,   2.88151768e-05,
          8.09063067e-07,   4.78333859e-05,   3.67561347e-06,
          1.83302916e-06,   4.61068354e-04,   5.56491723e-05,
          1.22480

In [22]:
# check score for baseline
from scipy.ndimage import imread
sil_img = imread('preprocessing/sil_img.png')
sil_img = np.zeros_like(())
sil_img = np.stack([sil_img for _ in range(x_test[ind].shape[0])], axis =0)
preds = model.predict(np.expand_dims(sil_img,0))
pred_label = np.argmax(preds)
least_label = np.argmin(preds)
print("true label is: ",None, "predict label is :", pred_label )
print("true label is: ",None, "least label is :", least_label )
print("probabilities: ",preds)

('true label is: ', None, 'predict label is :', 4)
('true label is: ', None, 'least label is :', 9)
('probabilities: ', array([[  2.00639246e-04,   3.77109903e-03,   1.10510550e-03,
          2.19969079e-03,   4.05168533e-01,   2.57756021e-02,
          1.08790089e-04,   5.47663367e-04,   9.76997311e-04,
          9.78086609e-05,   5.66967530e-04,   2.24525778e-04,
          2.19173473e-03,   1.09841265e-01,   4.64367418e-04,
          2.24779658e-02,   2.96632352e-04,   6.42060826e-04,
          5.98546118e-04,   3.80103855e-04,   4.64846846e-04,
          8.08832177e-04,   5.78096658e-02,   7.96143548e-04,
          1.78918970e-04,   3.57979443e-04,   5.80873247e-03,
          1.45526283e-04,   9.15632397e-03,   1.86662108e-03,
          2.10869163e-02,   2.61320807e-02,   3.13818455e-03,
          5.20374393e-04,   3.71339060e-02,   3.27321119e-04,
          1.13273323e-01,   7.69588351e-03,   8.82721797e-04,
          4.33972627e-02,   4.45768441e-04,   5.73119968e-02,
          1.

In [16]:
from keras.preprocessing import image
from matplotlib import pylab as plt
def show_image(image, grayscale = True, ax=None, title=''):
    if ax is None:
        plt.figure()
    plt.axis('off')
    
    if len(image.shape) == 2 or grayscale == True:
        if len(image.shape) == 3:
            image = np.sum(image, axis=2)
            
        vmax = np.percentile(image, 99)
        vmin = np.min(image)

        plt.imshow(image, cmap=plt.cm.gray, vmin=vmin, vmax=vmax)
        plt.title(title)
    else:
        image = image + 127.5
        image = image.astype('uint8')
        
        plt.imshow(image)
        plt.title(title)
    

In [17]:
# visualization - Saliency Map
from vis.saliency import GradientSaliency
vanilla = GradientSaliency(model,true_label)
least_vanilla = GradientSaliency(model,least_label)
mask_saliency = vanilla.get_mask(x_test[ind])
least_mask_saliency = least_vanilla.get_mask(x_test[ind])

In [18]:
# visualization - guided backpropagation
from vis.guided_backprop import GuidedBackprop
guided_bprop = GuidedBackprop(model,true_label)
least_guided_bprop = GuidedBackprop(model,least_label)
mask = guided_bprop.get_mask(x_test[ind])
least_mask = least_guided_bprop.get_mask(x_test[ind])

dense_2/Softmax:0
dense_2/Softmax:0


In [ ]:
from vis.integrated_gradients import IntegratedGradients
inter_grad = IntegratedGradients(model,true_label)
least_inter_grad = IntegratedGradients(model,least_label)
mask_integrated = inter_grad.GetMask(x_test[ind], input_baseline = sil_img, nsamples=200)
least_mask_integrated = least_inter_grad.GetMask(x_test[ind], input_baseline = sil_img, nsamples=200)

In [ ]:
plt.figure(figsize=(20,8))

for i in range(7):
    show_image(mask[i], ax=plt.subplot(6,7,1+i), title='GuidedBackprop')
    show_image(least_mask[i], ax=plt.subplot(6,7,8+i), title='GuidedBackprop')
for i in range(7):
    show_image(mask_saliency[i], ax=plt.subplot(6,7,15+i), title='GradientSaliency')
    show_image(least_mask_saliency[i], ax=plt.subplot(6,7,22+i), title='GradientSaliency')
for i in range(7):
    show_image(mask_integrated[i], ax=plt.subplot(6,7,29+i), title='IntegratedGradients')
    show_image(least_mask_integrated[i], ax=plt.subplot(6,7,36+i), title='IntegratedGradients')